# Match Stat Scraper

In [3]:
#Scraping Imports
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
#SQL Imports
import mysql.connector
#Pandas imports
import pandas as pd

In [4]:
#DB Connection
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  passwd="NYg1@nts",
  database="NRL_data"
)
mycursor = mydb.cursor(buffered=True)

ProgrammingError: 1045 (28000): Access denied for user 'root'@'localhost' (using password: YES)

## Get all URL's of matches with missing data

In [56]:
#Get all matches
all_matches_query = pd.read_sql_query('SELECT id, date, url, home_team_id, away_team_id FROM Matches;', mydb)
all_match_df = pd.DataFrame(all_matches_query, columns=['id', 'date', 'url', 'home_team_id', 'away_team_id'])

#Find matches that were already scraped
already_scraped = 'SELECT DISTINCT match_id FROM PlayerMatchStats;'
mycursor.execute(already_scraped,)
results = mycursor.fetchall()
already_scraped_list = list(map(lambda x: x[0], results))

#Remove matches which were already scraped and save remaining match info to not_yet_scraped_df
not_yet_scraped = set(list(all_match_df['id'])) - set(already_scraped_list)
not_yet_scraped_df = all_match_df[all_match_df['id'].isin(not_yet_scraped)]
print(not_yet_scraped_df)

In [ ]:
#Instantiate a blank dictionary called scraping_dict
scraping_dict = {}

#Convert 'date' column of not yet_scraped_df to datetime
not_yet_scraped_df['date'] = pd.to_datetime(not_yet_scraped_df['date'])

#Populate scraping_dict with key = year and values = matches for that year
scraping_dict = {}
for year in list(not_yet_scraped_df['date'].dt.year.unique()):
    scraping_dict[year] = not_yet_scraped_df[not_yet_scraped_df['date'].dt.year == year]
#scraping_dict[2018]


Checking for mac64 chromedriver:81.0.4044.138 in cache
Driver found in /Users/nickpowers/.wdm/chromedriver/81.0.4044.138/mac64/chromedriver


## Create functions to identify team and players by id

In [ ]:
def find_team_id(name):
    find_team_query = 'SELECT id FROM Teams WHERE nickname = %s;'
    mycursor.execute(find_team_query, (name,))
    return mycursor.fetchone()[0]

def find_or_create_player(first_name, last_name, team_id):
    find_player_query = 'SELECT id FROM Players WHERE first_name = %s AND last_name LIKE %s AND current_team = %s LIMIT 1;'
    mycursor.execute(find_player_query, (first_name, '%' + last_name + '%', team_id))
    result = mycursor.fetchone()
    if result is None:
        insert_player_query = 'INSERT INTO Players (first_name, last_name, current_team) VALUES (%s, %s, %s);'
        data = (first_name, last_name, team_id)
        #print(data)
        mycursor.execute(insert_player_query, data)
        mydb.commit()
        result = find_or_create_player(first_name, last_name, team_id)
        return int(result)
    else:
        result = result[0]
        #print('id = ' + str(result))
        return result

## Create dictionary for saving results by season

In [ ]:
#Create dictionary of seasons
player_match_stats = dict.fromkeys(scraping_dict.keys(), {})

# 2018 Scraping

In [71]:
#Set Up WebDriver
driver = webdriver.Chrome(ChromeDriverManager().install())
wait = WebDriverWait(driver, 10)


Checking for mac64 chromedriver:81.0.4044.138 in cache
Driver found in /Users/nickpowers/.wdm/chromedriver/81.0.4044.138/mac64/chromedriver


In [60]:
for year in [2018]:
#for year in player_match_stats.keys():
    year_dict = {}
    year_dict['errors'] = []
    for match in scraping_dict[year].iterrows():
        match = match[1]
        match_key = match['url'].split(str(year) + '/')[1][:-1]
        for char in ['-vs-', '-v-', '/', '-']:
            match_key = match_key.replace(char, '_')
        year_dict[match_key] = {}
        
        try:
            print(match['url'])
            driver.get(match['url'])
            #home_xpath_div = '1', away_xpath_div = '2'
            for xpath in ['1', '2']:
                wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="player-stats"]/div[' + xpath + ']/div/div[3]/div/table/tbody')))
                for i in range(1, 18):
                    name_field = driver.find_element_by_xpath('//*[@id="player-stats"]/div[' + xpath + ']/div/div[3]/div/table/tbody/tr['+ str(i) +']/td[2]/a').get_attribute('innerText').strip()
                    first_name = name_field.split(' ')[0].strip().capitalize()
                    last_name = name_field.split(' ')[-1].strip().capitalize()
                    middle_name = name_field.split(' ')[-2].strip()
                    if middle_name.isalpha():
                        last_name = middle_name.capitalize() + ' ' + last_name
                    if xpath == '1':
                        team_id = match['home_team_id']
                    elif xpath == '2':
                        team_id = match['away_team_id']
                    full_name = first_name + '_' + last_name + '_' + str(team_id)
                    player_id = find_or_create_player(first_name, last_name, str(team_id))
                    
                    player_stat_list = []
                    player_stat_list.append(player_id)
                    player_stat_list.append(team_id)
                    player_stat_list.append(match['id'])
                    
                    for column in range(3, 67):
                        if column in [5, 7, 15, 17, 21, 34, 40, 47, 56, 64]:
                            continue
                        else:
                            stat_field = driver.find_element_by_xpath('//*[@id="player-stats"]/div[' + xpath + ']/div/div[3]/div/table/tbody/tr[' + str(i) + ']/td[' + str(column) + ']')
                            player_stat_list.append(stat_field.get_attribute('innerText').strip())
                    print(player_stat_list)
                    year_dict[match_key][full_name] = player_stat_list
        except:
            print('error: ' + match['url'])
            year_dict['errors'].append(match['url'])
        print(year_dict[match_key])
    
    csv_data = pd.DataFrame.from_dict(year_dict, orient='index').replace('-', 0)
    #csv_data = pd.DataFrame.from_dict(year_dict, orient='index', columns=column_names).replace('-', 0)
    csv_data = csv_data.replace({pd.np.nan: 0})

    print(csv_data)
    print(year_dict)
    player_match_stats[year] = year_dict

https://www.nrl.com/draw/nrl-premiership/2018/finals-week-1/storm-vs-rabbitohs/
[568, 7, 462, '1', 'Fullback', '80:00', '-', '-', '-', '-', '-', '-', '-', '28', '13', '105', '50', '38', '1', '1', '1', '-', '2', '-', '11', '3.19s', '3', '15', '-', '1', '-', '21', '36', '1.62', '33.3%', '2', '2', '2', '-', '3', '-', '-', '-', '-', '-', '-', '-', '-', '1', '1', '-', '-', '-', '-', '-', '80:00', '-']
[185, 7, 462, '2', 'Winger', '80:00', '8', '2', '-', '-', '-', '-', '-', '26', '8', '89', '25', '29', '1', '-', '-', '-', '-', '-', '5', '4.39s', '-', '-', '-', '2', '-', '1', '13', '0.13', '57.1%', '4', '3', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '2', '1', '-', '-', '-', '-', '-', '80:00', '-']
[623, 7, 462, '3', 'Centre', '80:00', '8', '2', '-', '-', '-', '-', '-', '51', '14', '118', '-', '27', '1', '1', '-', '-', '4', '1', '11', '3.03s', '-', '-', '-', '-', '-', '1', '16', '0.07', '83.3%', '10', '2', '-', '-', '-', '1', '17', '-', '-', '1', '-', '-', '-', '-', '-', '-', '-',

{'Billy_Slater_7': [568, 7, 462, '1', 'Fullback', '80:00', '-', '-', '-', '-', '-', '-', '-', '28', '13', '105', '50', '38', '1', '1', '1', '-', '2', '-', '11', '3.19s', '3', '15', '-', '1', '-', '21', '36', '1.62', '33.3%', '2', '2', '2', '-', '3', '-', '-', '-', '-', '-', '-', '-', '-', '1', '1', '-', '-', '-', '-', '-', '80:00', '-'], 'Suliasi_Vunivalu_7': [185, 7, 462, '2', 'Winger', '80:00', '8', '2', '-', '-', '-', '-', '-', '26', '8', '89', '25', '29', '1', '-', '-', '-', '-', '-', '5', '4.39s', '-', '-', '-', '2', '-', '1', '13', '0.13', '57.1%', '4', '3', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '2', '1', '-', '-', '-', '-', '-', '80:00', '-'], 'Cheyse_Blair_7': [623, 7, 462, '3', 'Centre', '80:00', '8', '2', '-', '-', '-', '-', '-', '51', '14', '118', '-', '27', '1', '1', '-', '-', '4', '1', '11', '3.03s', '-', '-', '-', '-', '-', '1', '16', '0.07', '83.3%', '10', '2', '-', '-', '-', '1', '17', '-', '-', '1', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '80

https://www.nrl.com/draw/nrl-premiership/2018/finals-week-1/panthers-vs-warriors/
[316, 12, 463, '1', 'Fullback', '80:00', '-', '-', '-', '-', '-', '-', '-', '55', '20', '189', '77', '44', '1', '-', '1', '-', '7', '-', '15', '3.62s', '-', '-', '-', '1', '-', '12', '29', '0.6', '66.7%', '4', '1', '1', '-', '3', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '80:00', '-']
[323, 12, 463, '2', 'Winger', '80:00', '-', '-', '-', '-', '-', '-', '-', '32', '21', '180', '37', '70', '-', '-', '-', '-', '2', '1', '18', '3.34s', '3', '21', '-', '-', '-', '6', '24', '0.29', '80%', '4', '-', '1', '-', '2', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '80:00', '-']
[326, 12, 463, '3', 'Centre', '80:00', '-', '-', '-', '-', '-', '-', '-', '10', '5', '41', '-', '5', '-', '-', '-', '-', '3', '-', '4', '2.93s', '2', '22', '-', '-', '-', '5', '8', '1', '52.9%', '9', '6', '2', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '1', '-'

{'Dallin_Watene-zelezniak_12': [316, 12, 463, '1', 'Fullback', '80:00', '-', '-', '-', '-', '-', '-', '-', '55', '20', '189', '77', '44', '1', '-', '1', '-', '7', '-', '15', '3.62s', '-', '-', '-', '1', '-', '12', '29', '0.6', '66.7%', '4', '1', '1', '-', '3', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '80:00', '-'], 'Josh_Mansour_12': [323, 12, 463, '2', 'Winger', '80:00', '-', '-', '-', '-', '-', '-', '-', '32', '21', '180', '37', '70', '-', '-', '-', '-', '2', '1', '18', '3.34s', '3', '21', '-', '-', '-', '6', '24', '0.29', '80%', '4', '-', '1', '-', '2', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '80:00', '-'], 'Waqa_Blake_12': [326, 12, 463, '3', 'Centre', '80:00', '-', '-', '-', '-', '-', '-', '-', '10', '5', '41', '-', '5', '-', '-', '-', '-', '3', '-', '4', '2.93s', '2', '22', '-', '-', '-', '5', '8', '1', '52.9%', '9', '6', '2', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '1', '-', '-', '-', '

https://www.nrl.com/draw/nrl-premiership/2018/finals-week-1/roosters-vs-sharks/
[432, 15, 464, '1', 'Fullback', '80:00', '-', '-', '-', '-', '-', '-', '-', '67', '23', '240', '76', '76', '-', '3', '2', '-', '8', '1', '22', '3.59s', '5', '47', '-', '-', '-', '8', '32', '0.35', '-', '-', '1', '-', '-', '5', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '80:00', '-']
[431, 15, 464, '2', 'Winger', '80:00', '4', '1', '-', '-', '-', '-', '-', '44', '21', '191', '38', '78', '1', '-', '1', '-', '3', '1', '21', '2.7s', '1', '11', '-', '-', '-', '6', '30', '0.29', '100%', '1', '-', '-', '-', '2', '-', '-', '-', '-', '-', '-', '-', '-', '2', '1', '-', '-', '-', '-', '-', '80:00', '-']
[486, 15, 464, '3', 'Centre', '80:00', '8', '1', '1', '4', '1', '40%', '-', '50', '10', '77', '-', '33', '-', '-', '-', '-', '6', '-', '7', '3.33s', '1', '5', '-', '-', '-', '1', '12', '0.1', '92.3%', '12', '-', '1', '-', '-', '1', '48', '-', '-', '-', '-', '-', '-', '-', '-', '-', '3', 

...

{'Darius_Boyd_1': [22, 1, 465, '1', 'Fullback', '80:00', '4', '1', '-', '-', '-', '-', '-', '41', '12', '107', '40', '16', '1', '-', '-', '-', '6', '-', '4', '3.16s', '2', '23', '-', '-', '1', '20', '27', '1.67', '62.5%', '5', '2', '1', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '80:00', '-'], 'Corey_Oates_1': [20, 1, 465, '2', 'Winger', '80:00', '-', '-', '-', '-', '-', '-', '-', '15', '10', '94', '43', '47', '-', '-', '-', '-', '2', '-', '10', '2.84s', '-', '-', '-', '-', '-', '-', '12', '-', '80%', '4', '-', '1', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '2', '2', '-', '-', '-', '-', '-', '80:00', '-'], 'James_Roberts_1': [27, 1, 465, '3', 'Centre', '80:00', '-', '-', '-', '-', '-', '-', '-', '46', '8', '103', '-', '15', '-', '-', '-', '-', '8', '-', '7', '3.18s', '-', '-', '-', '-', '-', '1', '8', '0.13', '88.9%', '16', '2', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '80:00', '-'], 'Jor

https://www.nrl.com/draw/nrl-premiership/2018/finals-week-2/sharks-vs-panthers/
error: https://www.nrl.com/draw/nrl-premiership/2018/finals-week-2/sharks-vs-panthers/
{}
https://www.nrl.com/draw/nrl-premiership/2018/finals-week-2/rabbitohs-vs-dragons/
error: https://www.nrl.com/draw/nrl-premiership/2018/finals-week-2/rabbitohs-vs-dragons/
{}
https://www.nrl.com/draw/nrl-premiership/2018/finals-week-3/storm-vs-sharks/
error: https://www.nrl.com/draw/nrl-premiership/2018/finals-week-3/storm-vs-sharks/
{}
https://www.nrl.com/draw/nrl-premiership/2018/finals-week-3/roosters-vs-rabbitohs/
error: https://www.nrl.com/draw/nrl-premiership/2018/finals-week-3/roosters-vs-rabbitohs/
{}
https://www.nrl.com/draw/nrl-premiership/2018/grand-final/roosters-vs-storm/
[432, 15, 470, '1', 'Fullback', '80:00', '-', '-', '-', '-', '-', '-', '-', '37', '17', '214', '99', '53', '-', '1', '1', '-', '1', '-', '18', '3.45s', '1', '5', '-', '-', '2', '6', '25', '0.35', '100%', '7', '-', '-', '-', '1', '-', '-', 

{'James_Tedesco_15': [432, 15, 470, '1', 'Fullback', '80:00', '-', '-', '-', '-', '-', '-', '-', '37', '17', '214', '99', '53', '-', '1', '1', '-', '1', '-', '18', '3.45s', '1', '5', '-', '-', '2', '6', '25', '0.35', '100%', '7', '-', '-', '-', '1', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '1', '-', '-', '-', '80:00', '-'], 'Daniel_Tupou_15': [431, 15, 470, '2', 'Winger', '80:00', '4', '1', '-', '-', '-', '-', '-', '44', '19', '184', '38', '37', '1', '-', '-', '-', '4', '-', '17', '3.2s', '1', '9', '-', '-', '-', '-', '20', '-', '75%', '3', '1', '-', '-', '1', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '80:00', '-'], 'Latrell_Mitchell_15': [486, 15, 470, '3', 'Centre', '80:00', '12', '1', '1', '3', '3', '66.67%', '-', '46', '10', '59', '-', '10', '1', '-', '-', '-', '4', '-', '6', '3.79s', '-', '-', '-', '1', '-', '7', '18', '0.7', '84.6%', '11', '2', '-', '-', '2', '1', '12', '-', '-', '-', '-', '-', '-', '-', '1', '-', '1', '-', '-', '-',

                                                                                 0   \
errors                            https://www.nrl.com/draw/nrl-premiership/2018/...   
finals_week_1_storm_rabbitohs                                        Billy_Slater_7   
finals_week_1_panthers_warriors                          Dallin_Watene-zelezniak_12   
finals_week_1_roosters_sharks                                      James_Tedesco_15   
finals_week_1_broncos_dragons                                         Darius_Boyd_1   
finals_week_2_sharks_panthers                                                     0   
finals_week_2_rabbitohs_dragons                                                   0   
finals_week_3_storm_sharks                                                        0   
finals_week_3_roosters_rabbitohs                                                  0   
grand_final_roosters_storm                                         James_Tedesco_15   

                                          

[10 rows x 34 columns]
{'errors': ['https://www.nrl.com/draw/nrl-premiership/2018/finals-week-2/sharks-vs-panthers/', 'https://www.nrl.com/draw/nrl-premiership/2018/finals-week-2/rabbitohs-vs-dragons/', 'https://www.nrl.com/draw/nrl-premiership/2018/finals-week-3/storm-vs-sharks/', 'https://www.nrl.com/draw/nrl-premiership/2018/finals-week-3/roosters-vs-rabbitohs/'], 'finals_week_1_storm_rabbitohs': {'Billy_Slater_7': [568, 7, 462, '1', 'Fullback', '80:00', '-', '-', '-', '-', '-', '-', '-', '28', '13', '105', '50', '38', '1', '1', '1', '-', '2', '-', '11', '3.19s', '3', '15', '-', '1', '-', '21', '36', '1.62', '33.3%', '2', '2', '2', '-', '3', '-', '-', '-', '-', '-', '-', '-', '-', '1', '1', '-', '-', '-', '-', '-', '80:00', '-'], 'Suliasi_Vunivalu_7': [185, 7, 462, '2', 'Winger', '80:00', '8', '2', '-', '-', '-', '-', '-', '26', '8', '89', '25', '29', '1', '-', '-', '-', '-', '-', '5', '4.39s', '-', '-', '-', '2', '-', '1', '13', '0.13', '57.1%', '4', '3', '-', '-', '-', '-', '-', '

## Manually update error URLs

In [62]:
errors = player_match_stats[2018]['errors']
##player_match_stats[2018].pop('errors')

KeyError: 'errors'

In [63]:
#Get match ids of errored URLs for manual updating
find_incorrect_url_matches = 'SELECT id, url FROM Matches WHERE url IN {};'.format(tuple(errors))
mycursor.execute(find_incorrect_url_matches,)
results = mycursor.fetchall()
print(results)

[(466, 'https://www.nrl.com/draw/nrl-premiership/2018/finals-week-2/sharks-vs-panthers/'), (467, 'https://www.nrl.com/draw/nrl-premiership/2018/finals-week-2/rabbitohs-vs-dragons/'), (468, 'https://www.nrl.com/draw/nrl-premiership/2018/finals-week-3/storm-vs-sharks/'), (469, 'https://www.nrl.com/draw/nrl-premiership/2018/finals-week-3/roosters-vs-rabbitohs/')]


In [64]:
mycursor.executemany("UPDATE Matches SET url = %s WHERE id = %s",
                    [("https://www.nrl.com/draw/nrl-premiership/2018/finals-week-2/game-1/", "466"),
                    ("https://www.nrl.com/draw/nrl-premiership/2018/finals-week-2/game-12/", "467"),
                    ("https://www.nrl.com/draw/nrl-premiership/2018/finals-week-3/game-1/", "468"),
                    ("https://www.nrl.com/draw/nrl-premiership/2018/finals-week-3/game-12/", "469")])
mydb.commit()

## Run scraping code for updated match URLs

In [69]:
errored_match_query = pd.read_sql_query('SELECT id, date, url, home_team_id, away_team_id FROM Matches WHERE id IN (466, 467, 468, 469);', mydb)
errored_match_df = pd.DataFrame(errored_match_query, columns=['id', 'date', 'url', 'home_team_id', 'away_team_id'])

In [72]:
for match in errored_match_df.iterrows():
        match = match[1]
        match_key = match['url'].split(str(year) + '/')[1][:-1]
        for char in ['-vs-', '-v-', '/', '-']:
            match_key = match_key.replace(char, '_')
        year_dict[match_key] = {}
        
        try:
            print(match['url'])
            driver.get(match['url'])
            #home_xpath_div = '1', away_xpath_div = '2'
            for xpath in ['1', '2']:
                wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="player-stats"]/div[' + xpath + ']/div/div[3]/div/table/tbody')))
                for i in range(1, 18):
                    name_field = driver.find_element_by_xpath('//*[@id="player-stats"]/div[' + xpath + ']/div/div[3]/div/table/tbody/tr['+ str(i) +']/td[2]/a').get_attribute('innerText').strip()
                    first_name = name_field.split(' ')[0].strip().capitalize()
                    last_name = name_field.split(' ')[-1].strip().capitalize()
                    middle_name = name_field.split(' ')[-2].strip()
                    if middle_name.isalpha():
                        last_name = middle_name.capitalize() + ' ' + last_name
                    if xpath == '1':
                        team_id = match['home_team_id']
                    elif xpath == '2':
                        team_id = match['away_team_id']
                    full_name = first_name + '_' + last_name + '_' + str(team_id)
                    player_id = find_or_create_player(first_name, last_name, str(team_id))
                    
                    player_stat_list = []
                    player_stat_list.append(player_id)
                    player_stat_list.append(team_id)
                    player_stat_list.append(match['id'])
                    
                    for column in range(3, 67):
                        if column in [5, 7, 15, 17, 21, 34, 40, 47, 56, 64]:
                            continue
                        else:
                            stat_field = driver.find_element_by_xpath('//*[@id="player-stats"]/div[' + xpath + ']/div/div[3]/div/table/tbody/tr[' + str(i) + ']/td[' + str(column) + ']')
                            player_stat_list.append(stat_field.get_attribute('innerText').strip())
                    print(player_stat_list)
                    year_dict[match_key][full_name] = player_stat_list
        except:
            print('error: ' + match['url'])
            year_dict['errors'].append(match['url'])
        print(year_dict[match_key])

https://www.nrl.com/draw/nrl-premiership/2018/finals-week-2/game-1/
[472, 4, 466, '1', 'Fullback', '80:00', '12', '1', '2', '3', '2', '80%', '-', '37', '6', '96', '40', '6', '1', '-', '-', '-', '2', '-', '6', '3.5s', '-', '-', '-', '-', '-', '21', '30', '3.5', '66.7%', '2', '1', '-', '-', '2', '2', '11', '-', '-', '1', '-', '-', '1', '-', '-', '-', '-', '-', '-', '-', '80:00', '-']
[97, 4, 466, '2', 'Winger', '80:00', '-', '-', '-', '-', '-', '-', '-', '35', '13', '131', '14', '37', '1', '-', '-', '-', '5', '1', '13', '3.7s', '-', '-', '-', '-', '-', '1', '14', '0.08', '100%', '3', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '80:00', '-']
[471, 4, 466, '4', 'Centre', '80:00', '-', '-', '-', '-', '-', '-', '-', '31', '10', '98', '-', '45', '-', '1', '-', '-', '2', '1', '8', '3.54s', '-', '-', '-', '1', '-', '3', '12', '0.3', '93.3%', '14', '1', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '80

{'Valentine_Holmes_4': [472, 4, 466, '1', 'Fullback', '80:00', '12', '1', '2', '3', '2', '80%', '-', '37', '6', '96', '40', '6', '1', '-', '-', '-', '2', '-', '6', '3.5s', '-', '-', '-', '-', '-', '21', '30', '3.5', '66.7%', '2', '1', '-', '-', '2', '2', '11', '-', '-', '1', '-', '-', '1', '-', '-', '-', '-', '-', '-', '-', '80:00', '-'], 'Sosaia_Feki_4': [97, 4, 466, '2', 'Winger', '80:00', '-', '-', '-', '-', '-', '-', '-', '35', '13', '131', '14', '37', '1', '-', '-', '-', '5', '1', '13', '3.7s', '-', '-', '-', '-', '-', '1', '14', '0.08', '100%', '3', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '80:00', '-'], 'Ricky_Leutele_4': [471, 4, 466, '4', 'Centre', '80:00', '-', '-', '-', '-', '-', '-', '-', '31', '10', '98', '-', '45', '-', '1', '-', '-', '2', '1', '8', '3.54s', '-', '-', '-', '1', '-', '3', '12', '0.3', '93.3%', '14', '1', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '80:00', '

https://www.nrl.com/draw/nrl-premiership/2018/finals-week-2/game-12/
[346, 13, 467, '1', 'Fullback', '80:00', '-', '-', '-', '-', '-', '-', '-', '32', '11', '129', '48', '39', '1', '-', '-', '-', '4', '-', '9', '2.39s', '2', '14', '-', '-', '-', '16', '28', '1.45', '100%', '3', '-', '-', '-', '3', '1', '6', '-', '-', '1', '-', '-', '-', '1', '-', '-', '-', '-', '-', '-', '80:00', '-']
[348, 13, 467, '2', 'Winger', '80:00', '-', '-', '-', '-', '-', '-', '-', '14', '9', '99', '57', '33', '-', '-', '-', '-', '1', '-', '7', '3.23s', '-', '-', '-', '-', '-', '-', '9', '-', '88.9%', '8', '1', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '2', '2', '1', '-', '-', '-', '-', '80:00', '-']
[352, 13, 467, '3', 'Centre', '80:00', '-', '-', '-', '-', '-', '-', '-', '23', '12', '103', '-', '44', '-', '-', '-', '-', '1', '-', '11', '2.64s', '2', '17', '-', '-', '-', '12', '22', '1', '84.2%', '16', '2', '1', '-', '1', '1', '21', '-', '-', '-', '-', '1', '-', '1', '1', '-', '1', '-', '-', '-',

{'Alex_Johnston_13': [346, 13, 467, '1', 'Fullback', '80:00', '-', '-', '-', '-', '-', '-', '-', '32', '11', '129', '48', '39', '1', '-', '-', '-', '4', '-', '9', '2.39s', '2', '14', '-', '-', '-', '16', '28', '1.45', '100%', '3', '-', '-', '-', '3', '1', '6', '-', '-', '1', '-', '-', '-', '1', '-', '-', '-', '-', '-', '-', '80:00', '-'], 'Campbell_Graham_13': [348, 13, 467, '2', 'Winger', '80:00', '-', '-', '-', '-', '-', '-', '-', '14', '9', '99', '57', '33', '-', '-', '-', '-', '1', '-', '7', '3.23s', '-', '-', '-', '-', '-', '-', '9', '-', '88.9%', '8', '1', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '2', '2', '1', '-', '-', '-', '-', '80:00', '-'], 'Greg_Inglis_13': [352, 13, 467, '3', 'Centre', '80:00', '-', '-', '-', '-', '-', '-', '-', '23', '12', '103', '-', '44', '-', '-', '-', '-', '1', '-', '11', '2.64s', '2', '17', '-', '-', '-', '12', '22', '1', '84.2%', '16', '2', '1', '-', '1', '1', '21', '-', '-', '-', '-', '1', '-', '1', '1', '-', '1', '-', '-', '-', '80:0

https://www.nrl.com/draw/nrl-premiership/2018/finals-week-3/game-1/
[568, 7, 468, '1', 'Fullback', '80:00', '8', '2', '-', '-', '-', '-', '-', '78', '19', '189', '74', '14', '2', '-', '1', '-', '11', '-', '12', '4.69s', '2', '12', '-', '1', '5', '18', '39', '0.95', '33.3%', '1', '1', '1', '-', '3', '1', '-', '-', '-', '-', '-', '1', '-', '2', '2', '-', '1', '-', '-', '-', '80:00', '-']
[185, 7, 468, '2', 'Winger', '80:00', '-', '-', '-', '-', '-', '-', '-', '11', '6', '53', '10', '17', '-', '-', '-', '-', '1', '-', '3', '4.51s', '-', '-', '-', '1', '-', '-', '7', '-', '81.8%', '9', '2', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '1', '2', '-', '1', '-', '-', '-', '80:00', '-']
[187, 7, 468, '3', 'Centre', '80:00', '-', '-', '-', '-', '-', '-', '-', '20', '10', '76', '-', '17', '-', '-', '-', '-', '2', '-', '9', '3.15s', '2', '15', '-', '-', '-', '-', '13', '-', '76.2%', '16', '3', '2', '-', '-', '2', '26', '-', '-', '2', '-', '-', '-', '2', '2', '1', '-', '-', '-', '-', '80

{'Billy_Slater_7': [568, 7, 468, '1', 'Fullback', '80:00', '8', '2', '-', '-', '-', '-', '-', '78', '19', '189', '74', '14', '2', '-', '1', '-', '11', '-', '12', '4.69s', '2', '12', '-', '1', '5', '18', '39', '0.95', '33.3%', '1', '1', '1', '-', '3', '1', '-', '-', '-', '-', '-', '1', '-', '2', '2', '-', '1', '-', '-', '-', '80:00', '-'], 'Suliasi_Vunivalu_7': [185, 7, 468, '2', 'Winger', '80:00', '-', '-', '-', '-', '-', '-', '-', '11', '6', '53', '10', '17', '-', '-', '-', '-', '1', '-', '3', '4.51s', '-', '-', '-', '1', '-', '-', '7', '-', '81.8%', '9', '2', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '1', '2', '-', '1', '-', '-', '-', '80:00', '-'], 'Will_Chambers_7': [187, 7, 468, '3', 'Centre', '80:00', '-', '-', '-', '-', '-', '-', '-', '20', '10', '76', '-', '17', '-', '-', '-', '-', '2', '-', '9', '3.15s', '2', '15', '-', '-', '-', '-', '13', '-', '76.2%', '16', '3', '2', '-', '-', '2', '26', '-', '-', '2', '-', '-', '-', '2', '2', '1', '-', '-', '-', '-', '80:00', 

https://www.nrl.com/draw/nrl-premiership/2018/finals-week-3/game-12/
[432, 15, 469, '1', 'Fullback', '80:00', '-', '-', '-', '-', '-', '-', '-', '29', '15', '154', '48', '45', '-', '1', '1', '-', '2', '-', '15', '3.79s', '1', '14', '-', '-', '-', '7', '23', '0.47', '100%', '1', '-', '-', '-', '2', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '1', '-', '-', '-', '80:00', '-']
[431, 15, 469, '2', 'Winger', '80:00', '4', '1', '-', '-', '-', '-', '-', '41', '19', '144', '50', '54', '1', '-', '-', '-', '3', '-', '17', '3.33s', '-', '-', '-', '1', '-', '-', '21', '-', '71.4%', '5', '1', '1', '-', '3', '-', '-', '-', '-', '-', '-', '-', '-', '1', '2', '-', '-', '-', '-', '-', '80:00', '-']
[417, 15, 469, '3', '2nd Row', '80:00', '-', '-', '-', '-', '-', '-', '-', '32', '6', '52', '2', '24', '-', '-', '-', '-', '-', '4', '6', '3.91s', '1', '8', '-', '-', '-', '-', '6', '-', '89.5%', '34', '3', '1', '-', '1', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '1', '-', '-', '-', '

{'James_Tedesco_15': [432, 15, 469, '1', 'Fullback', '80:00', '-', '-', '-', '-', '-', '-', '-', '29', '15', '154', '48', '45', '-', '1', '1', '-', '2', '-', '15', '3.79s', '1', '14', '-', '-', '-', '7', '23', '0.47', '100%', '1', '-', '-', '-', '2', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '1', '-', '-', '-', '80:00', '-'], 'Daniel_Tupou_15': [431, 15, 469, '2', 'Winger', '80:00', '4', '1', '-', '-', '-', '-', '-', '41', '19', '144', '50', '54', '1', '-', '-', '-', '3', '-', '17', '3.33s', '-', '-', '-', '1', '-', '-', '21', '-', '71.4%', '5', '1', '1', '-', '3', '-', '-', '-', '-', '-', '-', '-', '-', '1', '2', '-', '-', '-', '-', '-', '80:00', '-'], 'Mitchell_Aubusson_15': [417, 15, 469, '3', '2nd Row', '80:00', '-', '-', '-', '-', '-', '-', '-', '32', '6', '52', '2', '24', '-', '-', '-', '-', '-', '4', '6', '3.91s', '1', '8', '-', '-', '-', '-', '6', '-', '89.5%', '34', '3', '1', '-', '1', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '1', '-', '-', '-', '80:

In [49]:
column_names = ['player_id', 'team_id', 'match_id', 'number', 'position', 'minutes_played', 'points', 'tries',
                'conversions','conversion_attempts', 'penalty_goals', 'conversion_percentage','field_goals',
                'fantasy_points', 'total_runs', 'total_run_metres', 'kick_return_metres', 'post_contact_metres',
                'line_breaks', 'line_break_assists', 'try_assists', 'line_engaged_runs', 'tackle_breaks', 'hit_ups',
                'play_the_ball', 'average_play_the_ball_seconds', 'dummy_half_runs', 'dummy_half_run_metres', 
                'steals', 'offloads', 'dummy_passes', 'passes', 'receipts', 'pass_to_run_ratio', 'tackle_percentage',
                'tackles_made', 'tackles_missed', 'ineffective_tackles', 'intercepts', 'kicks_defused', 'kicks',
                'kicking_metres', 'forced_drop_outs', 'bomb_kicks', 'grubbers', 'fourty_twenty',
                'cross_field_kicks', 'kicked_dead', 'errors', 'handling_errors', 'one_on_ones_lost', 'penalties',
                'on_report', 'sin_bins', 'send_offs', 'stint_one', 'stint_two']

year_df = pd.DataFrame()
for match in year_dict.keys():
    match_df = pd.DataFrame.from_dict(year_dict[match], orient='index', columns=column_names).reset_index()
    year_df = year_df.append(match_df, ignore_index = True)
year_df = year_df.replace('-', 0).replace({pd.np.nan: 0})
year_df.to_csv('./csv_files/' + str(year) + '_data.csv')

